In [1]:
!pip install -q pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y > /dev/null 2>&1
!pip install -q torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install -q unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from unsloth import FastLanguageModel

MAX_SEQ_LENGTH = 2048
LORA_RANK = 16
DTYPE = None
LOAD_IN_4BIT = False

model, tokenizer = FastLanguageModel.from_pretrained(
    "/kaggle/input/cs-vietnamese/pytorch/default/2/cs_model",
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=LOAD_IN_4BIT,
    max_lora_rank=LORA_RANK,
    gpu_memory_utilization=0.8,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_RANK,
    lora_alpha=LORA_RANK,
    lora_dropout=0,
    bias = "none",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "up_proj",
        "down_proj", "o_proj", "gate_proj"],
    use_rslora=False,
    use_gradient_checkpointing="unsloth",
    random_state = 42,
    loftq_config = None
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-23 13:16:28.284471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750684588.499956      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750684588.562590      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.5: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Unsloth 2025.6.5 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


In [3]:
import re
import pandas as pd
from datasets import Dataset

reasoning_start = "<thinking>"
reasoning_end = "</thinking>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

system_prompt = f"""Bạn được cung cấp một đoạn mã.
Hãy phân tích đoạn mã và trình bày quá trình suy luận của bạn.
Hãy đặt quá trình suy luận giữa {reasoning_start} và {reasoning_end}.
Sau đó, hãy đưa ra kết quả cuối cùng (output) giữa {solution_start}{solution_end}"""

df = pd.read_csv("/kaggle/input/cs-qas/code_with_output_train.csv")
df = df[['new_code', 'output']].dropna()

dataset = Dataset.from_pandas(df)
dataset = dataset.map(lambda x: {
    "prompt": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": x["new_code"]},
    ],
    "output": x["output"]
})

Map:   0%|          | 0/3245 [00:00<?, ? examples/s]

In [4]:
############# Reward for correct FORMAT #############

pattern = re.compile(
    rf"^[\s]{{0,}}"
    rf"{reasoning_start}.+?{reasoning_end}.*?"
    rf"{solution_start}(.+?){solution_end}"
    rf"[\s]{{0,}}\$",
    flags=re.MULTILINE | re.DOTALL
)

def match_format_exactly(completions, **kwargs):
    completion_contents = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, content) for content in completion_contents]
    return [3.0 if match else 0.0 for match in matches]

def match_format_approximately(completions, **kwargs):
    tags = [reasoning_start, reasoning_end, solution_start, solution_end]
    return [
        sum(0.5 if response.count(tag) == 1 else -1.0 for tag in tags)
        for completion in completions
        for response in [completion[0]["content"]]
    ]

############# Reward for correct OUTPUT #############

import io
import sys

def execute_code(code):
    old_stdout = sys.stdout
    sys.stdout = io.StringIO()
    try:
        exec(code)
        output = sys.stdout.getvalue().strip()
    except Exception as e:
        output = str(e)
    finally:
        sys.stdout = old_stdout
    return output

def check_answer(prompts, completions, **kwargs):
    responses = [completion[0]["content"] for completion in completions]
    extracted_responses = [
        guess.group(1)
        if (guess := pattern.search(r)) is not None else None
        for r in responses
    ]
    true_answers = [execute_code(prompt) for prompt in prompts]
    
    scores = []
    for guess, true_answer in zip(extracted_responses, true_answers):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        if guess == true_answer:
            score += 3.0
        elif guess.strip() == true_answer.strip():
            score += 1.5
        else:
            score -= 1.5
        scores.append(score)
    return scores

In [5]:
from unsloth import is_bfloat16_supported
from trl import GRPOConfig, GRPOTrainer

max_len = max(dataset.map(
    lambda x: {"tokens": tokenizer.apply_chat_template(
    x["prompt"], add_generation_prompt=True, tokenize=True)},
    batched=True,
).map(lambda x: {"length": len(x["tokens"])})["length"])

max_prompt_length = max_len + 1

training_args = GRPOConfig(
    per_device_train_batch_size = 16,
    gradient_accumulation_steps = 4,
    num_generations = 8,
    warmup_steps = 5,
    num_train_epochs = 1,
    learning_rate = 5e-6,
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 5e-4,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir="checkpoint/reasoning",
    report_to = "none",
    remove_unused_columns=True,
    max_prompt_length=max_prompt_length,
    max_completion_length=MAX_SEQ_LENGTH - max_prompt_length,
    max_steps = -1,
    save_steps = 250,
    max_grad_norm = 0.1,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
)

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        match_format_exactly,
        match_format_approximately,
        check_answer
    ],
    args=training_args,
    train_dataset=dataset,
)

Map:   0%|          | 0/3245 [00:00<?, ? examples/s]

Map:   0%|          | 0/3245 [00:00<?, ? examples/s]

In [6]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,245 | Num Epochs = 1 | Total steps = 203
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 4 x 1) = 128
 "-____-"     Trainable parameters = 11,272,192/1,247,086,592 (0.90% trained)
`generation_config` default values have been modified to match model-specific defaults: {'max_length': 131072, 'temperature': 0.6, 'top_p': 0.9}. If this is not desired, please set these values explicitly.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std
1,0.000000,-3.859375,0.295914,373.328125,138.000000,778.000000,0.140625,307.109070,138.000000,709.000000,0.009185,0.000000,0.000000,-3.859375,0.546778,0.000000,0.000000
2,0.000000,-4.000000,0.000000,778.000000,778.000000,778.000000,1.000000,0.000000,0.000000,0.000000,0.035140,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000
3,0.000000,-4.000000,0.000000,778.000000,778.000000,778.000000,1.000000,0.000000,0.000000,0.000000,0.032636,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000
4,0.000000,-4.000000,0.000000,778.000000,778.000000,778.000000,1.000000,0.000000,0.000000,0.000000,0.036433,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000
5,0.000000,-4.000000,0.000000,778.000000,778.000000,778.000000,1.000000,0.000000,0.000000,0.000000,0.040882,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000
6,0.000000,-4.000000,0.000000,778.000000,778.000000,778.000000,1.000000,0.000000,0.000000,0.000000,0.029615,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000
7,0.000000,-4.000000,0.000000,778.000000,778.000000,778.000000,1.000000,0.000000,0.000000,0.000000,0.021364,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000
8,0.000000,-4.000000,0.000000,778.000000,778.000000,778.000000,1.000000,0.000000,0.000000,0.000000,0.020961,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000
9,0.000000,-4.000000,0.000000,778.000000,778.000000,778.000000,1.000000,0.000000,0.000000,0.000000,0.020148,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000
10,0.000000,-4.000000,0.000000,778.000000,778.000000,778.000000,1.000000,0.000000,0.000000,0.000000,0.017689,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.06 GiB. GPU 0 has a total capacity of 14.74 GiB of which 94.12 MiB is free. Process 3790 has 14.62 GiB memory in use. Of the allocated memory 14.35 GiB is allocated by PyTorch, and 120.62 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model.save_pretrained("reasoning_cs")